En este documento vamos a hacer los cambios requeridos en xml: 

    Cambiar el numero de gender por el nombre correspondiente
    
    Eliminar el "Level 0"

In [3]:
import re
import os
import xml.etree.ElementTree as ET

tree = ET.parse('data_xml.xml')
root = tree.getroot()


xml_list = []
for child in root:
    xml_dict = {}
    for subchild in child:
        xml_dict[subchild.tag] = subchild.text
    xml_list.append(xml_dict) # Es una lista que contiene diccionarios

In [4]:
#Hacemos la primera prueba, cogiendo solo un elemento del fichero 
xml_list [0]['gender'] = "Man"

In [5]:
type(xml_list)

list

In [4]:
#Nuestro fichero es una lista de diccionarios. 
#Hemos realizado una función iterando por los elementos de nuestra lista (que son diccionarios) y cambiando el value de la key "gender"
#Hemos identificado una columna redundante, que era "Level0" y hemos utilizado un bucle for tambien para que lo elimine en todos los diccionarios.

def limpieza_xml (lista):
    for diccionario in lista:

        if diccionario['gender'] == "0" :
            diccionario["gender"] = "Man"
        elif diccionario["gender"] == "1":
            diccionario["gender"] = "Woman"
        elif diccionario['gender'] == "2":
            diccionario['gender'] = "Nonbinary"
        elif diccionario['gender'] == "3":
            diccionario['gender'] = "Prefer not to say"
        elif diccionario['gender'] == "4":
            diccionario['gender'] = "Prefer to self-describe"
    
    for diccionario in lista:
        diccionario.pop("level_0")

    return lista
    


In [5]:
xml_limpio = limpieza_xml(xml_list)

In [9]:
#Hemos definido la funcion sacar_values donde iteramos por cada uno de los elementos de la lista xml_list, transformando estos values 
#a una tupla y apendeandola en la lista de values.

def sacar_values(xml_list): 
    values = []
    for i in xml_list:
        values.append(tuple(i.values()))
    return values


In [10]:
values_xml = sacar_values(xml_list)
values_xml

[('1', '784', '50-54', 'Man'),
 ('2', '924', '22-24', 'Man'),
 ('3', '575', '45-49', 'Man'),
 ('4', '781', '45-49', 'Man'),
 ('5', '1020', '25-29', 'Woman'),
 ('6', '141', '18-21', 'Woman'),
 ('7', '484', '30-34', 'Man'),
 ('8', '1744', '22-24', 'Man'),
 ('9', '655', '30-34', 'Man'),
 ('10', '1777', '40-44', 'Man'),
 ('11', '3081', '18-21', 'Woman'),
 ('12', '1922', '18-21', 'Woman'),
 ('13', '852', '45-49', 'Man'),
 ('14', '838', '22-24', 'Man'),
 ('15', '563', '35-39', 'Man'),
 ('16', '1315', '50-54', 'Man'),
 ('17', '479', '18-21', 'Man'),
 ('18', '249', '22-24', 'Man'),
 ('19', '650', '30-34', 'Man'),
 ('20', '1461', '70+', 'Man'),
 ('21', '551', '25-29', 'Woman'),
 ('22', '258', '30-34', 'Man'),
 ('23', '773', '35-39', 'Man'),
 ('24', '630', '18-21', 'Man'),
 ('25', '2467', '25-29', 'Woman'),
 ('26', '7750', '45-49', 'Man'),
 ('27', '607', '22-24', 'Man'),
 ('28', '525', '22-24', 'Woman'),
 ('29', '501', '18-21', 'Man'),
 ('30', '415', '22-24', 'Man'),
 ('31', '317', '30-34', 'Man

In [13]:
#Hemos definido la funcion importar_datos donde le pasamos el parametro xml_list definido anteriormente. Este parametro es una lista de tuplas.
#El metodo se encarga de importar los datos mediante el metodo INSERT INTO a la tabla_xml.
#Hemos elegido executemany debido a la longitud del texto, y hemos llamado a la funcion para que se ejecute.

def importar_datos (xml_list):
    import mysql.connector
    cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='proyecto1')
    mycursor = cnx.cursor()

    sql = "INSERT INTO tabla_xml (index_xml,time,age,gender) VALUES (%s,%s,%s,%s)"
    val = xml_list
    try:
        mycursor.executemany(sql, val)
        cnx.commit()
        print(mycursor.rowcount, "registro/s insertado/s.")
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [12]:
importar_datos(values_xml)

25972 registro/s insertado/s.
